# Step4: Stock Selection based on Stock Price Prediction Results

---
**What does this notebook do?**
- After step3 builds up several machine learning models for four periods and record the results
- This notebook will first analyze the model prediction accuracy performance 
- Then will select the top-performing models to use for stock price prediction
- Finally it will select the top 5 stocks from each prediction model per period
- The selected 5 stocks will be constructed together to form the portfolio

**To-Do List**
1. Select the r2 score and mean absolute error columns from each period file
2. Data analysis the model accuracy and find out the top-performing models 
3. Select the predicted future price by top-performing models, and the last day price
4. Compare the last day price and the preidcted future price, to select the top 5 stocks

**Feature Name for Comparision**
- Model Accuracy: `score`, `mean_absolute_error`
- Last day price & Future Price: `last_day_value`, `future_price`

---

In [1]:
import pandas as pd
import numpy as np

In [2]:
def output_compare_table(time_period_results, compare_value): 
    """
    This function selects the specific time period file 
    stored in Results/StockPrediction
    and get all columns including needed compare value
    """
    import pandas as pd
    file_loaction = '../Results/StockPrediction/' + time_period_results + '.csv'
    dataset = pd.read_csv(file_loaction)
    
    output_list = ['stock_code']
    for column in dataset.columns.tolist():
        if (compare_value in column): output_list.append(column)
    
    return dataset[output_list]

# Analysis for Model Prediction Accuracy Scores (r2 Score + Mean Absolute Error)


In [3]:
# Output the Prediction Model Performance for all time
output_compare_table('all_time_results', 'score')\
    .merge(output_compare_table('all_time_results', 'mean_absolute_error'))\
    .to_csv('../Results/StockPrediction/[prediction]-[all_time]-[accuracy].csv', index=False)

# Output the Prediction Model Performance for covid time
output_compare_table('covid_time_results', 'score')\
    .merge(output_compare_table('covid_time_results', 'mean_absolute_error'))\
    .to_csv('../Results/StockPrediction/[prediction]-[covid_time]-[accuracy].csv', index=False)

# Output the Prediction Model Performance for pre covid time
output_compare_table('pre_covid_time_results', 'score')\
    .merge(output_compare_table('pre_covid_time_results', 'mean_absolute_error'))\
    .to_csv('../Results/StockPrediction/[prediction]-[per_covid_time]-[accuracy].csv', index=False)

# Output the Prediction Model Performance for pre covid test time
output_compare_table('pre_covid_test_time_results', 'score')\
    .merge(output_compare_table('pre_covid_test_time_results', 'mean_absolute_error'))\
    .to_csv('../Results/StockPrediction/[prediction]-[pre_covid_test_time]-[accuracy].csv', index=False)

In [4]:
def get_model_performance_mean_median(time_period_results, compare_value): 
    """
    This function returns a table containing the mean and median of 
    a certain period, with the breakdown of each prediction model
    """
    df = output_compare_table(time_period_results, compare_value)
    mean_df = pd.DataFrame(df.mean(numeric_only=True).to_dict(),index=['mean'])
    mean_df = mean_df.T

    df = output_compare_table(time_period_results, compare_value)
    median_df = pd.DataFrame(df.median(numeric_only=True).to_dict(),index=['median'])
    median_df = median_df.T

    mean_median_df = mean_df.join(median_df).reset_index()
    mean_median_df = mean_median_df.rename(columns = {'index':'performance metrics'})
    
    return mean_median_df

In [5]:
get_model_performance_mean_median('all_time_results', 'score')

,performance metrics,mean,median
0,linear_regression: score,0.272129,0.419874
1,decision_tree: score,-1.196054,-1.102049
2,random_forest: score,-0.745515,-0.579106
3,support_vector_machine_linear: score,0.375408,0.465029
4,support_vector_machine_poly: score,-3.981019,-0.761395
5,support_vector_machine_rbf: score,-6.038996,-0.752960
6,lstm: score,0.945613,0.954663


In [6]:
def get_four_period_model_performance_mean_median(compare_value):
    """
    This function builds on the get_model_performance_mean_median,
    it combines the four period of the mean and median 
    of all predictions models 
    """
    # Get the all time period compare value results
    df1 = get_model_performance_mean_median('all_time_results', compare_value)
    df1['time period'] = 'all time'
    # Get the covid time period compare value results
    df2 = get_model_performance_mean_median('covid_time_results', compare_value)
    df2['time period'] = 'covid time'
    # Get the pre covid time period compare value results
    df3 = get_model_performance_mean_median('pre_covid_time_results', compare_value)
    df3['time period'] = 'pre covid time'
    # Get the pre covid test time period compare value results
    df4 = get_model_performance_mean_median('pre_covid_test_time_results', compare_value)
    df4['time period'] = 'pre covid test time'
    
    # Concat all four time periods together and reset index
    summary = pd.concat([df1, df2, df3, df4])
    return summary.set_index(['time period', 'performance metrics'])

In [7]:
# Save the r2 score summary for four periods
get_four_period_model_performance_mean_median('score').to_excel('../Results/StockPrediction/[prediction]-[four period]-[score].xlsx')
get_four_period_model_performance_mean_median('score').to_csv('../Results/StockPrediction/[prediction]-[four period]-[score].csv')
get_four_period_model_performance_mean_median('score')

mean    median
time period         performance metrics                                      
all time            linear_regression: score               0.272129  0.419874
                    decision_tree: score                  -1.196054 -1.102049
                    random_forest: score                  -0.745515 -0.579106
                    support_vector_machine_linear: score   0.375408  0.465029
                    support_vector_machine_poly: score    -3.981019 -0.761395
                    support_vector_machine_rbf: score     -6.038996 -0.752960
                    lstm: score                            0.945613  0.954663
covid time          linear_regression: score              -2.828694 -1.652856
                    decision_tree: score                 -10.000772 -5.058511
                    random_forest: score                  -6.250970 -3.376103
                    support_vector_machine_linear: score  -3.026539 -1.687897
                    support_vector_machine_poly: score    -4.589191 -2.005635
                    support_vector_machine_rbf: score     -9.796238 -4.392865
                    lstm: score                            0.470422  0.598973
pre covid time      linear_regression: score               0.076040  0.166927
                    decision_tree: score                  -1.904856 -0.723941
                    random_forest: score                  -1.151915 -0.490967
                    support_vector_machine_linear: score   0.140689  0.216569
                    support_vector_machine_poly: score    -3.156300 -1.461041
                    support_vector_machine_rbf: score     -6.754086 -2.073172
                    lstm: score                            0.925183  0.934373
pre covid test time linear_regression: score              -4.733497 -2.527090
                    decision_tree: score                  -9.638987 -4.781196
                    random_forest: score                  -7.906879 -2.862657
                    support_vector_machine_linear: score  -4.103128 -1.886447
                    support_vector_machine_poly: score    -4.949788 -1.984549
                    support_vector_machine_rbf: score    -14.155775 -7.306961
                    lstm: score                            0.474123  0.602501

In [8]:
# Save the mean absoulte summary for four periods
get_four_period_model_performance_mean_median('mean_absolute_error').to_excel('../Results/StockPrediction/[prediction]-[four period]-[mean_absolute_error].xlsx')
get_four_period_model_performance_mean_median('mean_absolute_error').to_csv('../Results/StockPrediction/[prediction]-[four period]-[mean_absolute_error].csv')
get_four_period_model_performance_mean_median('mean_absolute_error')

mean  \
time period         performance metrics                                             
all time            linear_regression: mean_absolute_error               7.286629   
                    decision_tree: mean_absolute_error                  20.043445   
                    random_forest: mean_absolute_error                  19.668776   
                    support_vector_machine_linear: mean_absolute_error   7.154203   
                    support_vector_machine_poly: mean_absolute_error    31.321402   
                    support_vector_machine_rbf: mean_absolute_error     30.783246   
                    lstm: mean_absolute_error                            2.445631   
covid time          linear_regression: mean_absolute_error               7.630205   
                    decision_tree: mean_absolute_error                  11.227872   
                    random_forest: mean_absolute_error                  10.473761   
                    support_vector_machine_linear: mean_absolute_error   8.073570   
                    support_vector_machine_poly: mean_absolute_error     9.330573   
                    support_vector_machine_rbf: mean_absolute_error     12.954777   
                    lstm: mean_absolute_error                            2.535814   
pre covid time      linear_regression: mean_absolute_error               4.261601   
                    decision_tree: mean_absolute_error                   7.250351   
                    random_forest: mean_absolute_error                   6.698439   
                    support_vector_machine_linear: mean_absolute_error   4.355446   
                    support_vector_machine_poly: mean_absolute_error     9.537662   
                    support_vector_machine_rbf: mean_absolute_error     11.780266   
                    lstm: mean_absolute_error                            1.293047   
pre covid test time linear_regression: mean_absolute_error               3.927093   
                    decision_tree: mean_absolute_error                   5.790588   
                    random_forest: mean_absolute_error                   5.006539   
                    support_vector_machine_linear: mean_absolute_error   3.888833   
                    support_vector_machine_poly: mean_absolute_error     4.026281   
                    support_vector_machine_rbf: mean_absolute_error      6.727166   
                    lstm: mean_absolute_error                            1.355541   

                                                                          median  
time period         performance metrics                                           
all time            linear_regression: mean_absolute_error              3.267216  
                    decision_tree: mean_absolute_error                  5.674391  
                    random_forest: mean_absolute_error                  4.968980  
                    support_vector_machine_linear: mean_absolute_error  3.260732  
                    support_vector_machine_poly: mean_absolute_error    5.016201  
                    support_vector_machine_rbf: mean_absolute_error     5.878456  
                    lstm: mean_absolute_error                           0.905673  
covid time          linear_regression: mean_absolute_error              2.707085  
                    decision_tree: mean_absolute_error                  3.281775  
                    random_forest: mean_absolute_error                  2.746091  
                    support_vector_machine_linear: mean_absolute_error  2.555575  
                    support_vector_machine_poly: mean_absolute_error    2.722914  
                    support_vector_machine_rbf: mean_absolute_error     3.202845  
                    lstm: mean_absolute_error                           0.934031  
pre covid time      linear_regression: mean_absolute_error              2.379095  
                    decision_tree: mean_absolute_error                  4.767377  
                    random_fores

---
**Which Prediction Model to Choose?**
- From the data analsis shown above, we can find that 
- `linear regression model`, `support vector machine (linear) model` and `LSTM model` 
- have the best performance and highest accuracy scores in the all four periods
- thus we will then analysis their prediction prices in the next section
---

## Analysis of Historical Price & Future Price

In [9]:
def get_historical_future_price(time_period_results):
    """
    This functin returns the future value predicted by 
    linear regression, Support Vector Machine(linear) and LSTM Network,
    the average predicated value from these three models,
    and also the last day value of each stock in one period.
    """

    historical_price_day = output_compare_table(time_period_results, 'last_day_value')
    historical_price_day = historical_price_day[['stock_code', 'linear_regression: last_day_value']].rename(columns={'linear_regression: last_day_value': 'historical_price'})
    # historical_price_day.head()
    
    future_price = output_compare_table(time_period_results, 'future_price')[['stock_code', 'linear_regression: future_price', 'support_vector_machine_linear: future_price', 'lstm: future_price']]
    # future_price.head()
    
    summary = pd.merge(historical_price_day, future_price, on='stock_code', how='left')
    summary['mean_future_price'] = (summary['linear_regression: future_price'] + summary['support_vector_machine_linear: future_price'] + summary['lstm: future_price'])/3
    
    # Save the historical and future prices summary
    file_name = f'../Results/StockPrediction/[prediction]-[{time_period_results}]-[results]'
    summary.to_csv(f'{file_name}.csv', index=False)
    summary.to_excel(f'{file_name}.xlsx', index=False)
    
    return summary

In [10]:
get_historical_future_price('all_time_results')
get_historical_future_price('covid_time_results')
get_historical_future_price('pre_covid_time_results')
get_historical_future_price('pre_covid_test_time_results')

,stock_code,historical_price,linear_regression: future_price,support_vector_machine_linear: future_price,lstm: future_price,mean_future_price
0,HK.00700,361.839680,315.743079,331.299886,358.984611,335.342525
1,HK.00005,59.142820,57.949863,57.881569,59.100564,58.310665
2,HK.01299,79.134000,75.660608,75.482674,78.675499,76.606260
3,HK.00939,5.989420,5.538374,5.633283,5.907776,5.693144
4,HK.00388,237.150000,245.582309,233.690275,240.278730,239.850438
5,HK.02318,87.053850,86.781005,81.529250,85.800515,84.703590
6,HK.01398,5.392660,4.911350,5.046338,5.299193,5.085627
7,HK.00941,58.857000,59.562263,57.997020,57.707619,58.422301
8,HK.03968,37.232850,36.386038,34.616381,36.782666,35.928361
9,HK.00669,60.770000,52.968090,51.509551,59.007831,54.495157


In [11]:
def get_grow_stock_list(time_period_results, num_of_top):
    """
    This function returns certain number of top stocks of one certain period,
    with four prediction groups: linear regression, support vector machine(linear),
    LSTM Network, and the average predicted value
    """
    import pandas as pd
    pd.set_option('display.max_colwidth', None)
    
    df = get_historical_future_price(time_period_results)
    print(f"\n-------For {time_period_results}-------\n")
    print(f"There are {len(df)} stocks in {time_period_results}")

    # Have all grow stocks
    # mean_grow_stock_list = df[df['mean_future_price'] > df['historical_price']]['stock_code'].tolist()
    # linear_grow_stock_list = df[df['linear_regression: future_price'] > df['historical_price']]['stock_code'].tolist()
    # svm_grow_stock_list = df[df['support_vector_machine_linear: future_price'] > df['historical_price']]['stock_code'].tolist()
    # lstm_grow_stock_list = df[df['lstm: future_price'] > df['historical_price']]['stock_code'].tolist()
    
    def get_top_stocks(df, compare_column, num_of_top):
        df = df[df[compare_column] > df['historical_price']].copy()
        df['difference'] = df[compare_column] - df['historical_price']
        return df.sort_values(by=['difference'], ascending=False).head(num_of_top)['stock_code'].tolist()
    
    mean_grow_stock_list = get_top_stocks(df, 'mean_future_price', num_of_top)
    linear_grow_stock_list = get_top_stocks(df, 'linear_regression: future_price', num_of_top)
    svm_grow_stock_list = get_top_stocks(df, 'support_vector_machine_linear: future_price', num_of_top)
    lstm_grow_stock_list = get_top_stocks(df, 'lstm: future_price', num_of_top)
    
    print(f"There are {len(mean_grow_stock_list)} stocks by mean")
    print(f"There are {len(linear_grow_stock_list)} stocks by linear")
    print(f"There are {len(svm_grow_stock_list)} stocks by svm")
    print(f"There are {len(lstm_grow_stock_list)} stocks by lstm")

    portfolio_input = pd.DataFrame(columns=['time period', 'model', 'portfolio stock input'])
    portfolio_input.loc[len(portfolio_input.index)] = [time_period_results, 'Linear Regression', linear_grow_stock_list] 
    portfolio_input.loc[len(portfolio_input.index)] = [time_period_results, 'Support Vector Machine(linear)', svm_grow_stock_list] 
    portfolio_input.loc[len(portfolio_input.index)] = [time_period_results, 'LSTM Network', lstm_grow_stock_list] 
    portfolio_input.loc[len(portfolio_input.index)] = [time_period_results, 'Mean Average', mean_grow_stock_list] 

    return portfolio_input

In [12]:
portfolio_input_all_time = get_grow_stock_list('all_time_results', 5)
portfolio_input_covid_time = get_grow_stock_list('covid_time_results', 5)
portfolio_input_pre_covid_time = get_grow_stock_list('pre_covid_time_results', 5)
portfolio_input_pre_covid_test_time = get_grow_stock_list('pre_covid_test_time_results', 5)

portfolio_input_all_period = pd.concat([portfolio_input_all_time, portfolio_input_covid_time, 
                                        portfolio_input_pre_covid_time, portfolio_input_pre_covid_test_time])


-------For all_time_results-------

There are 39 stocks in all_time_results
There are 5 stocks by mean
There are 5 stocks by linear
There are 5 stocks by svm
There are 5 stocks by lstm

-------For covid_time_results-------

There are 39 stocks in covid_time_results
There are 5 stocks by mean
There are 5 stocks by linear
There are 5 stocks by svm
There are 5 stocks by lstm

-------For pre_covid_time_results-------

There are 39 stocks in pre_covid_time_results
There are 5 stocks by mean
There are 5 stocks by linear
There are 5 stocks by svm
There are 5 stocks by lstm

-------For pre_covid_test_time_results-------

There are 39 stocks in pre_covid_test_time_results
There are 5 stocks by mean
There are 5 stocks by linear
There are 5 stocks by svm
There are 5 stocks by lstm


In [13]:
# Display  all resutls in the cell
pd.set_option('display.max_colwidth', None)

portfolio_input_all_period = portfolio_input_all_period.set_index(['time period', 'model'])

portfolio_input_all_period.to_csv('../Results/StockPrediction/portfolio_input_all_period_top5.csv')
portfolio_input_all_period.to_excel('../Results/StockPrediction/portfolio_input_all_period_top5.xlsx')

portfolio_input_all_period

portfolio stock input
time period                 model                                                                             
all_time_results            Linear Regression               [HK.00700, HK.02020, HK.00669, HK.02313, HK.02331]
                            Support Vector Machine(linear)  [HK.00700, HK.02020, HK.01211, HK.02313, HK.00669]
                            LSTM Network                    [HK.01299, HK.00016, HK.01109, HK.00011, HK.00006]
                            Mean Average                    [HK.00700, HK.02020, HK.02313, HK.00669, HK.02331]
covid_time_results          Linear Regression               [HK.00700, HK.01211, HK.02020, HK.00388, HK.02331]
                            Support Vector Machine(linear)  [HK.01211, HK.00388, HK.00700, HK.02020, HK.02331]
                            LSTM Network                    [HK.00388, HK.02313, HK.02318, HK.02020, HK.02688]
                            Mean Average                    [HK.01211, HK.00700, HK.00388, HK.02020, HK.02331]
pre_covid_time_results      Linear Regression               [HK.00291, HK.00011, HK.02313, HK.01093, HK.01177]
                            Support Vector Machine(linear)  [HK.02020, HK.00291, HK.01093, HK.00011, HK.01177]
                            LSTM Network                    [HK.00700, HK.02318, HK.00002, HK.03968, HK.00005]
                            Mean Average                    [HK.00291, HK.01093, HK.00011, HK.02318, HK.01177]
pre_covid_test_time_results Linear Regression               [HK.00016, HK.00011, HK.00388, HK.02331, HK.01211]
                            Support Vector Machine(linear)  [HK.00011, HK.01211, HK.02331, HK.02020, HK.02388]
                            LSTM Network                    [HK.00011, HK.00388, HK.01211, HK.02020, HK.00002]
                            Mean Average                    [HK.00011, HK.01211, HK.02331, HK.00388, HK.00016]

- The top 5 stocks from each time period and each prediction model, will be allocated together to form the portfolio